# M1. Actividad
Alumno: Javier E. Agostini Castilla - A00827216

**Instrucciones**

Realizar una simulación en donde, a partir de variables iniciales como dimensiones de la habitación, porcentaje de celdas sucias y agentes limpiadores, con el paso del tiempo los agentes limpiadores comiencen a moverse por la habitación limpiando celdas sucias.


**Instalación de Librerias**

In [81]:
!pip3 install mesa

In [82]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid
from mesa.space import SingleGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

from mesa.space import ContinuousSpace


# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

**Declaración de Variables**

In [101]:
# Dimensiones de la habitacion
M = 8
N = 8

# Cantidad de agentes (limpiadores)
agent_qty = 10

# Porcentaje de celdas sucias (0-1)
dirty_cell_percentage = 0.5

# Tiempo maximo de ejecucion en segundos
max_exe_time = 5

**Desarrollo de Clases de Modelo y Agentes**

In [94]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Cleaner):
                grid[x][y] = 2
            else:
                grid[x][y] = content.live
    return grid

class Floor(Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        if np.random.choice(100) <= dirty_cell_percentage * 100:
            self.live = 1
        else:
            self.live = 0

class Cleaner(Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
                
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self,new_position)
        self.step

    def clean(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        if this_cell[0].live:
            this_cell[0].live = 0
            return True
        else:
            return False

    def step(self):
        if not self.clean():
            self.move()
  
class Room(Model):
  def __init__(self, m, n, agent_qty, dirty_cell_percentage, max_exe_time):
    self.agent_qty = m * n
    self.dirty_cell_percentage = dirty_cell_percentage
    self.clean_cell_percentage = 1 - dirty_cell_percentage
    self.start_time = time.time()
    self.max_exe_time = max_exe_time
    self.exe_time = None
    self.grid = MultiGrid(m, n, True)
    self.schedule = SimultaneousActivation(self)
    #self.running = True

    for (content, x, y) in self.grid.coord_iter():
            floor = Floor((x, y), self)
            self.grid.place_agent(floor, (x, y))
            self.schedule.add(floor)
        
        
    for i in range(agent_qty):
            cleaner = Cleaner(i,self)
            self.grid.place_agent(cleaner, (1, 1))
            self.schedule.add(cleaner)

    self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid},
            agent_reporters={'Move': lambda a: getattr(a, 'move', None)}
    )

  def count_dirty(self):
        count = 0
        for cell in self.grid.coord_iter():
            cell_content, x, y = cell
            if cell_content[0].live:
                count+=1
        return count

  def all_cells_clean(self):
      clean_cells = 0
      for cell in self.grid.coord_iter():
        cell_content, x, y = cell
        if cell_content[0].live:
          clean_cells = clean_cells + 1
      #return clean_cells
      
      self.clean_cell_percentage = clean_cells / (self.grid.width * self.grid.height)
      if self.clean_cell_percentage == 1:
        return True
      else: 
        return False

            
  def step(self):
        self.datacollector.collect(self)
        self.schedule.step()




  #   # Posicionar celdas sucias de forma aleatoria
  #   celdas_sucias = int((m * n) * por_celdas_sucias)
  #   lista_celdas_vacias = list(self.grid.empties)
  #   for celdas in range(celdas_sucias):
  #     celda_vacia = random.choice(lista_celdas_vacias)
  #     piso = Piso(celda_vacia, self)
  #     piso.estado = piso.SUCIO
  #     self.grid.place_agent(piso, celda_vacia)
  #     self.schedule.add(piso)
  #     lista_celdas_vacias.remove(celda_vacia)

  #   # Posicionar celdas limpias
  #   lista_celdas_vacias = kust(self.grid.empties)
  #   for celdas in lista_celdas_vacias:
  #     piso = Piso(celdas, self)
  #     self.grid.place_agent(piso, celdas)
  #     self.schedule.add(piso)
    
  #   # Posicionar agentes aspiradoras
  #   for i in range(num_agentes):
  #     aspiradora = Aspiradora(i, self)
  #     self.grid.place_agent (aspiradora, (1,1))
  #     self.schedule.add(aspiradora)
    
  #   self.colectordatos = DataCollector(
  #       model_reporters = {'Habitacion': obtener_habitacion},
  #       agent_reporters = {'Movimientos': lambda a: getattr(a, 'movimientos', None)}
  #   )
  
  # def todasceldaslimpias(self):
  #   celdas_limpias = 0
  #   for celda in self.grid.coord_iter():
  #     contenido_Celda, x, y = celda
  #     for contenido in contenido_celda:
  #       if isinstance(contenido, Piso) and contenido.estado == contenido.LIMPIO:
  #         celdas_limpias = celdas_limpias + 1
    
  #   self.por_celdas_limpias = celdas_limpias / (self.grid.width * self.grid.height)
  #   if self.por_celdas_limpias == 1:
  #     return True
  #   else: 
  #     return False
  
  # def step(self):
  #   if (time.time() - self.tiempo_inicio) < self.tiempo_limite or not modelo.todasceldaslimpias():
  #     self.colectordatos.collect(self)
  #     self.schedule.step()
  #   else:
  #     self.tiempo_ejecucion = str(datetime.timedelta(seconds = (time.time() - self.tiempo_inicio)))
  #     self.running = False

# class GameLifeAgent(Agent):
#     '''
#     Representa a un agente o una celda con estado vivo (1) o muerto (0)
#     '''
#     def __init__(self, unique_id, model):
#         '''
#         Crea un agente con estado inicial aleatorio de 0 o 1, también se le asigna un identificador 
#         formado por una tupla (x,y). También se define un nuevo estado cuyo valor será definido por las 
#         reglas mencionadas arriba.
#         '''
#         super().__init__(unique_id, model)
#         self.live = np.random.choice([0,1])
#         self.next_state = None
    
#     def step(self):
#         '''
#         Este método es el que calcula si la celda vivirá o morirá dependiendo el estado de sus vecinos.
#         El estado live de la siguiente generación no se cambia aquí se almacena en self.next_state. La idea 
#         es esperar a que todos los agentes calculen su estado y una vez hecho eso, ya hacer el cambio.
#         '''
#         live_neighbours = 0   
        
#         neighbours = self.model.grid.get_neighbors(
#             self.pos,
#             moore=True,
#             include_center=False)
        
#         for neighbor in neighbours:
#             live_neighbours = live_neighbours + neighbor.live
        
#         self.next_state = self.live
#         if self.next_state == 1:
#             if live_neighbours < 2 or live_neighbours > 3:
#                 self.next_state = 0
#         else:
#             if live_neighbours == 3:
#                 self.next_state = 1
    
#     def advance(self):
#         '''
#         Define el nuevo estado calculado del método step.
#         '''
#         self.live = self.next_state
            
# class GameLifeModel(Model):
#     '''
#     Define el modelo del juego de la vida.
#     '''
#     def __init__(self, width, height):
#         self.num_agents = width * height
#         self.grid = MultiGrid(width, height, True)
#         self.schedule = SimultaneousActivation(self)
        
#         for (content, x, y) in self.grid.coord_iter():
#             a = GameLifeAgent((x, y), self)
#             self.grid.place_agent(a, (x, y))
#             self.schedule.add(a)
        
#         # Aquí definimos con colector para obtener el grid completo.
#         self.datacollector = DataCollector(
#             model_reporters={"Grid": get_grid})
    
#     def step(self):
#         '''
#         En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
#         '''
#         self.datacollector.collect(self)
#         self.schedule.step()

**Ejecución**

In [102]:
start_time = time.time()
#tiempo_inicio = str(datetime.timedelta(seconds = max_exe_time))
model = Room(M, N, agent_qty, dirty_cell_percentage, max_exe_time)

steps = 0
while ((time.time() - start_time) < max_exe_time and not model.all_cells_clean()):
  model.step()
  steps += 1

exe_time = str(datetime.timedelta(seconds = (time.time() - start_time)))

print('Tiempo de ejecución:', exe_time)
print('Pasos tomados: ', steps)
print('Porcentaje de celdas limpias: ', (model.agent_qty - model.all_cells_clean()) / model.agent_qty * 100)

Tiempo de ejecución: 0:00:06.153081
Pasos tomados:  9762
Porcentaje de celdas limpias:  100.0


**Visualización**

In [86]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [87]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Reds')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [ ]:
anim

**Conclusión**

A pesar de que se me hizo compleja la actividad, considero que aprendí mucho sobre agentes y practiqué mis habilidades en Python. 

Pude observar que, al aumentar la cantidad de agentes, la cantidad de pasos necesarios para limpiar la habitación completamente, así como el tiempo de ejecución se disminuyen considerablemente. Es por eso que, simuladores como este, desarrollados en base a sistemas reales, permiten que las personas puedan experimentar y entender diversas situaciones y como ciertas acciones afectan el resultado final. 

Es decir, si se tratara de un negocio, podríamos graficar el impacto de la cantidad de agentes y los pasos o tiempo necesario para limpiar la habitación y encontrar la cantidad óptima de agentes considerando presupuesto u otros factores.